In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Chuẩn bị dữ liệu

In [16]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# data_dir = "../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
data_dir = "./dataset/asl_alphabet_train"

input_size= (128, 128) # Các ảnh sẽ được resize về một kích thước chung, độ phân giải càng cao càng tốn tài nguyên (CPU, GPU, RAM,..)
val_frac = 0.1         # Tỉ lệ của tập validation sẽ được chia ra từ tập train, thường chia theo các tỉ lệ 9:1, 8:2, 7:3
batch_size = 128       # Số ảnh xử lý/lần để cập nhật tham số, số càng cao càng tốn tài nguyên

# Tiền xử lý dữ liệu ảnh để mô hình học tốt hơn
data_augmentor = ImageDataGenerator(samplewise_center=True,            # Chuẩn hoá mean của các giá trị pixel
                                    samplewise_std_normalization=True, # Chuẩn hoá khoảng giá trị của các pixel
                                    validation_split=val_frac)

# Đọc dữ liệu từ thư mục và tiền xử lý
train_generator = data_augmentor.flow_from_directory(data_dir, 
                                                     target_size=input_size, 
                                                     batch_size=batch_size, 
                                                     shuffle=True, 
                                                     subset="training")
val_generator = data_augmentor.flow_from_directory(data_dir, 
                                                   target_size=input_size, 
                                                   batch_size=batch_size, 
                                                   subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [17]:
# Có 29 lớp trong tập train, tương ứng với 29 folders
train_generator.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 'del': 26,
 'nothing': 27,
 'space': 28}

In [18]:
len(train_generator.class_indices)

29

# Xây dựng mô hình

In [19]:
num_classes = len(train_generator.class_indices) # 29
input_shape = (128, 128, 3)                      # 3 chiều cho ảnh RGB

# Cấu trúc mô hình tham khảo
model = keras.models.Sequential([
    keras.Input(shape=input_shape),
    # block 1
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),     # Layer tích chập
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),     # Thêm Layer tích chập để học nhiều chi tiết hơn
    keras.layers.MaxPooling2D(pool_size=(2, 2)),                        # Layer MaxPool luôn theo sau layer Conv
    keras.layers.Dropout(0.5),                                          # Layer Dropout để mô hình không bị thiên kiến
    # block 2
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),     # Cấu trúc giống với block 1, tăng chiều từ 32 lên 64
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.5),
    # block 3
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),    # Cấu trúc giống với block 1, tăng chiều từ 64 lên 128
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),                         
    keras.layers.Dropout(0.5),
    # FCN block chịu trách nhiệm phân loại
    keras.layers.Flatten(),                                            # Chỉ có một layer flatten trong mô hình
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax'),             # Layer Dense cuối có chiều là tổng số class
])
model.compile(optimizer='adam',                 # Luôn sử dụng `adam`
              loss='categorical_crossentropy',  # `binary_crossentropy` cho lớp True/False, `categorical_crossentropy` cho nhiều lớp
              metrics=["accuracy"])             # Đánh giá mô hình dựa trên độ chính xác
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 124, 124, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 60, 60, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 58, 58, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 27, 27, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 25, 25, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │     9,437,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,794,109 (37.36 MB)

 Trainable params: 9,794,109 (37.36 MB)

 Non-trainable params: 0 (0.00 B)

# Huấn luyện mô hình


In [20]:
model.fit(train_generator, 
          epochs=2, 
          validation_data=val_generator)

Epoch 1/2
 68/612 ━━━━━━━━━━━━━━━━━━━━ 21:43 2s/step - accuracy: 0.0371 - loss: 3.3803

KeyboardInterrupt: 